# 科学技術計算 5 レポート


In [12]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag
rng = np.random.default_rng()

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow
from typing import Tuple, List, Optional, Union
from tqdm.auto import tqdm



## 課題05-1

In [13]:
#レイリー商反復について

#関数
def rayleigh_quotient_iteration(A: np.ndarray,mu: float, u, tol = 1e-12,n = np.ndarray, maxiter: int = 5000,) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    #入力 行列Aが対称かどうか muがシフト値uは
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n

    u_pre = u.copy()

    #レイリー商反復
    for i in range(maxiter):

        try:
            u = solve(A - mu * np.eye(n), u)    #シフト値muを用いて連立方程式を解く
        except:
            break
        
        u /= norm(u)    #正規化
        mu = u.T @ A @ u

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):         #uの値がめっちゃちかくなった→近似
            break
        u_pre = u.copy()
        
    return mu, u



In [14]:
#ハウスホルダー法
def householder_step( A: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2  #2次元であるかチェック
    assert A.shape[0] == A.shape[1] #正方行列かどうかチェック

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]  # Aのk列目のk+1行目以降の部分を取得(縦の部分)
    y = np.zeros_like(x) #全部ゼロの配列
    # y[0] = - np.sign(A[k + 1, k]) * norm(x)
    y[0] = - np.sign(x[0]) * norm(x)  # ベクトルyの先頭要素を設定

    # print("x", x)
    # print("y", y)

    u = (x - y) / norm(x - y)  # 単位ベクトルuを計算
    H = np.eye(len(x)) - 2 * np.outer(u, u)  # ハウスホルダー行列Hを計算

    Q = np.eye(n)   #単位行列
    Q[k + 1:, k + 1:] = H  # QにHを組み込む

    QAQ = Q @ A @ Q  # 変換後の行列を計算
    return QAQ, Q

def householder_transform(A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
                                       so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        # print(f"{k}-th step")
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q  # 直交行列を更新
        # print("Atri\n", Atri, sep='')
        # print("Q\n", Q, sep='')
        # print("Qhh\n", Qhh, sep='')
    return Atri, Qhh


In [15]:
#ぎぶんす変換
def givens_r_matrix(xp_xq: np.ndarray) -> np.ndarray:
    """generate a matrix of Givens rotation

    Args:
        xp_xq (ndarray): two values xp, xq

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(xp_xq) == 2

    xp, xq = xp_xq[0], xp_xq[1]

    # denominator = np.sqrt(xp**2 + xq**2)
    denominator = norm(xp_xq)
    c = xp / denominator
    s = -xq / denominator

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R

def qr_decom_by_givens(Atri_org: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        Atri_org (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(Atri_org.shape) == 2
    assert Atri_org.shape[0] == Atri_org.shape[1]
    # assert AAtri_org should be tridiagonal
    n = Atri_org.shape[0]

    Atri = Atri_org.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(Atri[[p, q]][:, p])
        Atri[[p, q]] = Rot2x2.T @ Atri[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = Atri
    return Q, R

In [16]:
#QR法関数
def eigh_by_qr_method(A0: np.ndarray, maxiter: int = 1000,) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        Tuple[np.ndarray, np.ndarray]: n-d vector of eigenvalues and orthogonal matrix with eigenvector columns
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    n = A0.shape[0]
    A = A0.copy()

    Qall = np.eye(n)
    diagonals_pre = diag(A)
    #ハウスホルダー変換
    Atri, Qhh = householder_transform(A)
    Qall = Qhh
    Atri_org = Atri.copy()

    for i in range(maxiter):
        
        Q, R = qr_decom_by_givens(Atri_org)
        Atri_org = R @ Q
        Qall = Qall @ Q

        diagonals = diag(Atri_org)
        # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        #     print(f"     diagonals {i:2d}", diagonals)
        #     print(f"superdiagonals {i:2d}", superdiagonals)

        if (np.allclose(diagonals, diagonals_pre)):
            # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
            #     print("A\n", A, sep='')
            break

        diagonals_pre = diagonals.copy()

    return diagonals, Qall


In [17]:
#今回のやつ
def eigh_via_qr_rayleigh( A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    # print("A =", A)
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    
    eigenvalues_approx, Qqr  = eigh_by_qr_method(A) #これでAがぎぶんす変換で更新しながら固有値と固有べくとるを出力できたってことかな

    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n, n))

    for k in range(n):
        mu0 = eigenvalues_approx[k]
        u0 = Qqr[k, :]


        mu_refined, u_refined = rayleigh_quotient_iteration(A, mu0, u0)

        eigenvalues[k] = mu_refined
        eigenvectors[:, k] = u_refined

    return eigenvalues, eigenvectors


In [18]:
#main
#実行
for i in tqdm(range(10)):
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(n, n))

    #対称行列
    A = A.T @ A
    print("行列 A:")
    print(A)

    d, U = eigh_via_qr_rayleigh(A)
    eigvals_np, eigvecs_np = np.linalg.eigh(A)


    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        d_np, U_np = np.linalg.eigh(A)
        U_np = U_np[:, ::-1]
        d_np = d_np[::-1]

        print("d == d_np", np.allclose(d, d_np))

        for i in range(len(U)):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )



  0%|          | 0/10 [00:00<?, ?it/s]

行列 A:
[[2.09844609 1.39705006 2.0524429  1.64216719 1.73963231 1.64673894
  2.35123948 1.7654544 ]
 [1.39705006 1.46231278 1.43986962 1.12880246 0.98260476 1.40140896
  1.88103201 1.27431553]
 [2.0524429  1.43986962 2.70268795 2.26617709 1.7454316  1.79151161
  2.81840352 2.03944719]
 [1.64216719 1.12880246 2.26617709 2.44656747 1.62583446 1.93339355
  2.6918849  1.54729493]
 [1.73963231 0.98260476 1.7454316  1.62583446 1.84781193 1.89366016
  2.30511165 1.74198249]
 [1.64673894 1.40140896 1.79151161 1.93339355 1.89366016 2.71327604
  3.01310481 1.89063958]
 [2.35123948 1.88103201 2.81840352 2.6918849  2.30511165 3.01310481
  4.3559268  2.57403137]
 [1.7654544  1.27431553 2.03944719 1.54729493 1.74198249 1.89063958
  2.57403137 2.56305473]]
 0 [-0.321840585648 -0.239705561416 -0.372850861647 -0.340638974444 -0.306879694430 -0.362504428465
 -0.492600068022 -0.340099648321]
 1 [-0.321840585648 -0.239705561416 -0.372850861647 -0.340638974444 -0.306879694430 -0.362504428465
 -0.49260006802

## 課題05-3

In [19]:
#QR分解を利用する関数 full/reduced QR分解を利用したthin SVDの計算方法
def svd_via_qr(A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    #行列のサイズ計算
    m, n = A.shape
    print("m =",m)
    print("n =",n)

    if(m >= n):
        print("nyaaaa1")
        #A.T @ A(対称行列)の対角化→固有値計算→QR文分解
        ATA = A.T @ A
        sigma, V = np.linalg.eigh(ATA)
        sigma = np.sqrt(np.maximum(sigma, 0))
        #V = n.n直交行列、sigma 対角行列
        V = V[:,::-1]
        sigma = sigma[::-1]
        a = A @ V
        U, c = np.linalg.qr(a)
        

    elif(m < n):
        print("nyaaaa2")
        AAT = A @ A.T
        sigma , U =  np.linalg.eigh(AAT)
        sigma = np.sqrt(np.maximum(sigma, 0))
        U = U[:,::-1]
        sigma = sigma[::-1]
        a = A.T @ U
        V ,c = np.linalg.qr(a)
        print("V = ",V)
        
    VT = V.T
    return U, sigma, VT

In [20]:
#固有値分解を利用した単純なthinSVDの計算方法
def svd_via_eigh( A: np.ndarray, ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    #行列のサイズ計算
    m, n = A.shape
    print("m =",m)
    print("n =",n)

    if(m >= n):
        print("nyaaaa1")
        #A.T @ A(対称行列)の対角化→固有値計算→QR文分解
        ATA = A.T @ A
        sigma, V = np.linalg.eigh(ATA)
        V = V[:,::-1] 
        sigma = np.sqrt(np.maximum(sigma, 0))
        sigma = sigma [::-1]
        Sigma_inv = np.diag(1.0 / sigma) # 各対角成分を逆数にするだけ
        print("sigma =",sigma)
        print("Sigma_inv =", Sigma_inv)
        
        U = A @ V @ Sigma_inv
    elif(m < n):
        print("nyaaaa2")
        AAT = A @ A.T
        sigma , U =  np.linalg.eigh(AAT)
        sigma = np.sqrt(np.maximum(sigma, 0))
        sigma = sigma [::-1]
        U = U[:,::-1]
        Sigma_inv = np.diag(1.0 / sigma) # 各対角成分を逆数にするだけ
        V = A.T @ U @ Sigma_inv
    VT = V.T
    return U ,sigma, VT

In [21]:
#実行

for i in tqdm(range(10)):
    m = rng.integers(low = 2, high = 10)
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(m, n))

    U, sigma, VT = svd_via_qr(A)


    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        U_np, sigma_np, VT_np = np.linalg.svd(A)
        
        print("sigma == sigma_np", np.allclose(sigma, sigma_np))

        for i in range(U.shape[1]):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )
        for i in range(len(VT)):
            print(
                f"column {i}: VT[{i}] == VT_np[{i}] ?",
                np.allclose(VT[i, :], VT_np[i, :]) or np.allclose(VT[i, :], -VT_np[i, :])
            )


for i in tqdm(range(10)):
    m = rng.integers(low = 2, high = 10)
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(m, n))

    U, sigma, VT = svd_via_eigh(A)
        

    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        U_np, sigma_np, VT_np = np.linalg.svd(A)
        
        print("sigma == sigma_np", np.allclose(sigma, sigma_np))

        for i in range(U.shape[1]):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )
        for i in range(len(VT)):
            print(
                f"column {i}: VT[{i}] == VT_np[{i}] ?",
                np.allclose(VT[i, :], VT_np[i, :]) or np.allclose(VT[i, :], -VT_np[i, :])
            )


  0%|          | 0/10 [00:00<?, ?it/s]

m = 3
n = 5
nyaaaa2
V =  [[-0.50486454  0.51698435 -0.20589889]
 [-0.66227914 -0.60254733 -0.04769583]
 [-0.18932625 -0.17825523 -0.69727356]
 [-0.34558063  0.5769385   0.00346012]
 [-0.3888801  -0.07092909  0.68492946]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: VT[1] == VT_np[1] ? True
column 2: VT[2] == VT_np[2] ? True
m = 5
n = 7
nyaaaa2
V =  [[-0.41993509  0.21784119  0.03164552  0.47599591  0.45925151]
 [-0.47149014 -0.28958806 -0.19082496  0.30975908 -0.56340996]
 [-0.3636743  -0.4678273  -0.35068145 -0.17994194 -0.04659484]
 [-0.35762911  0.44748129 -0.27166313 -0.70890902 -0.02736624]
 [-0.26669064  0.44295126  0.53380657  0.07374883 -0.51193963]
 [-0.4404077  -0.36204534  0.58965001 -0.26676083  0.37509778]
 [-0.2758825   0.34971335 -0.36492394  0.25679928  0.25682049]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? T

  0%|          | 0/10 [00:00<?, ?it/s]

m = 6
n = 5
nyaaaa1
sigma = [2.8773036  0.85811514 0.63166697 0.29775512 0.16777972]
Sigma_inv = [[0.34754761 0.         0.         0.         0.        ]
 [0.         1.16534479 0.         0.         0.        ]
 [0.         0.         1.58311269 0.         0.        ]
 [0.         0.         0.         3.35846455 0.        ]
 [0.         0.         0.         0.         5.96019606]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 3: U[3] == U_np[3] ? True
column 4: U[4] == U_np[4] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: VT[1] == VT_np[1] ? True
column 2: VT[2] == VT_np[2] ? True
column 3: VT[3] == VT_np[3] ? True
column 4: VT[4] == VT_np[4] ? True
m = 9
n = 2
nyaaaa1
sigma = [2.67362411 0.73595042]
Sigma_inv = [[0.37402416 0.        ]
 [0.         1.35878718]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: 